In [165]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [138]:
url='http://sh.bendibao.com/news/2022424/251365.shtm'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}

In [139]:
result=requests.get(url,headers=headers)

In [140]:
result

<Response [200]>

In [141]:
result.encoding='utf-8'

In [142]:
soup1=BeautifulSoup(result.text,'lxml')

In [143]:
soup2=soup1.find_all(class_='content') #alink = soup1.find_all("div", {"class":{"content"}})

In [144]:
soup=soup2[0].find_all('p')

In [147]:
#soup
a=soup[12].text

In [148]:
a[2:-1]

'病例485—病例556，居住于闵行区'

In [149]:
len(soup)

98

In [155]:
begin=[]
mid_begin=[]
end=[]

In [156]:
for x in range(len(soup)):
    result=soup[x]
    a=len(re.findall("[0-9]—病例",result.text))
    if(a!=0):
        b=result.text[2:-1]
        mid_begin.append(b)
    a=len(re.findall("[0-9]—无症状感染者",result.text))
    if(a!=0):
        b=result.text[2:-1]
        end.append(b)

In [159]:
print("mid_begin长度为",len(mid_begin),mid_begin[0],"mid_begin长度为",len(end),end[0])

mid_begin长度为 29 病例1—病例202，居住于浦东新区 mid_begin长度为 28 无症状感染者1—无症状感染者7026，居住于浦东新区


In [239]:
df1=pd.DataFrame(mid_begin,columns=['信息'])
df1=df1['信息'].str.replace('病例','').str.replace('居住于','').str.replace('—','，')
df1=df1.str.split('，',expand=True)

In [240]:
df2=pd.DataFrame(end,columns=['信息'])
df2=df2['信息'].str.replace('无症状感染者','').str.replace('居住于','').str.replace('—','，')
df2=df2.str.split('，',expand=True)

In [241]:
df1[3]=[int(x) for x in df1[0]]
df1[4]=[int(x) for x in df1[1]]

In [242]:
df2[3]=[int(x) for x in df2[0]]
df2[4]=[int(x) for x in df2[1]]

In [243]:
df1[5]=df1[4]-df1[3]

In [244]:
df2[5]=df2[4]-df2[3]

In [245]:
df1['状态']='确诊病例'
df2['状态']='无症状感染者'

In [246]:
df3=df1.append(df2)

In [247]:
df4=df3[['状态',2,5]]

In [248]:
df4.columns=['状态','居住地','人数']

In [250]:
df4.to_excel('2022424上海疫情爬虫数据.xlsx',encoding='utf-8')